In [1]:
from ase import Atoms
import ase.io
import ase.visualize
import numpy as np
from matplotlib import pyplot as plt

import openbabel

In [2]:
# set up table
class UFF_Decomposition():
    

IndentationError: expected an indented block (<ipython-input-2-f649107c2301>, line 3)

In [ ]:
atomic_energies_UFF = {'atom_id_babel':[], 'atomic_number':[], 'position':[], 'e_bond':[], 'e_angle':[], 'e_torsion':[], 'e_vdw':[], 'e_atomic':[]}

In [30]:
# def get_ob_properties(file_path, properties):
    
#     with open(file_path, 'r') as f:
#         file = f.readlines()
        
#     for p in properties:

def sort_dict(dict_unsorted):
    # sort by numerical key
    keys_sorted = list(dict_unsorted.keys())
    keys_sorted.sort()
    dict_sorted = dict()
    for ks in keys_sorted:
        dict_sorted[ks] = dict_unsorted[ks]
    return(dict_sorted)

def parse_positions(file):
    """
    get positions from output file
    """
    # find section for bond stretching because here I also write positions
    linenumber = 'Nan'
    for ln, line in enumerate(file):
        if line == 'B O N D   S T R E T C H I N G\n':
            linenumber = ln
            break

    # parse section for positions
    parse = file[linenumber+5:]
    i = 0

    position_dict = dict()
    while '     TOTAL BOND STRETCHING ENERGY' not in parse[i]:
        # get index and position of atom i
        idx_i = int(parse[i].strip('\n').split()[-8])
        position_i = [float(pos) for pos in parse[i].strip('\n').split()[-6:-3]]
        # sanity check that same index gives alsways same position
        if idx_i in position_dict.keys():
            assert position_dict[idx_i] == position_i, "index assigned to different positions"
        else:
            position_dict[idx_i] = position_i

        # get index and position of atom j
        idx_j = int(parse[i].strip('\n').split()[-7])
        position_j = [float(pos) for pos in parse[i].strip('\n').split()[-3:]]
        # sanity check that same index gives alsways same position
        if idx_j in position_dict.keys():
            assert position_dict[idx_j] == position_j, "index assigned to different positions"
        else:
            position_dict[idx_j] = position_j

        i += 1
    
    # sort by babel atom indices
    position_dict_sorted = sort_dict(position_dict)
    return(position_dict_sorted)


def parse_e_bond(file):
    """
    get bond energies from output file
    """
    # find section for bond stretching because here I also write positions
    linenumber = 'Nan'
    for ln, line in enumerate(file):
        if line == 'B O N D   S T R E T C H I N G\n':
            linenumber = ln
            break

    # parse section for positions
    parse = file[linenumber+5:]
    i = 0

    prop_dict = dict()
    while 'TOTAL BOND STRETCHING ENERGY' not in parse[i]:
        # get index of atom i
        idx_i = int(parse[i].strip('\n').split()[-8])
        # get index of atom j
        idx_j = int(parse[i].strip('\n').split()[-7])
        # get bond energy
        e_bond = float(parse[i].strip('\n').split()[7])
        
        # add half energy if index already exists otherwise initialize key and assign energy
        if idx_i in prop_dict.keys():
            prop_dict[idx_i] += e_bond/2
        else:
            prop_dict[idx_i] = e_bond/2
            
        if idx_j in prop_dict.keys():
            prop_dict[idx_j] += e_bond/2
        else:
            prop_dict[idx_j] = e_bond/2

        i += 1
    return(sort_dict(prop_dict))


def parse_e_angle(file):
    """
    get angle energies from output file
    """
    # find section for bond stretching because here I also write positions
    linenumber = 'Nan'
    for ln, line in enumerate(file):
        if line == 'A N G L E   B E N D I N G\n':
            linenumber = ln
            break

    # parse section for positions
    parse = file[linenumber+5:]
    i = 0

    prop_dict = dict()
    while 'TOTAL ANGLE BENDING ENERGY' not in parse[i]:
        # get atom indices
        idx_i = int(parse[i].strip('\n').split()[-3])
        idx_j = int(parse[i].strip('\n').split()[-2])
        idx_k = int(parse[i].strip('\n').split()[-1])
        
        # get angle energy
        e_angle = float(parse[i].strip('\n').split()[-4])
        
        # add 1/3 of energy if index already exists otherwise initialize key and assign energy
        if idx_i in prop_dict.keys():
            prop_dict[idx_i] += e_angle/3
        else:
            prop_dict[idx_i] = e_angle/3
            
        if idx_j in prop_dict.keys():
            prop_dict[idx_j] += e_angle/3
        else:
            prop_dict[idx_j] = e_angle/3
            
        if idx_k in prop_dict.keys():
            prop_dict[idx_k] += e_angle/3
        else:
            prop_dict[idx_k] = e_angle/3

        i += 1
    return(sort_dict(prop_dict))

def parse_e_torsional(file):
    """
    get torsional energies from output file
    """
    # find section for bond stretching because here I also write positions
    linenumber = 'Nan'
    for ln, line in enumerate(file):
        if line == 'T O R S I O N A L\n':
            linenumber = ln
            break

    # parse section for positions
    parse = file[linenumber+5:]
    i = 0

    prop_dict = dict()
    while 'TOTAL TORSIONAL ENERGY' not in parse[i]:
        # get atom indices
        idx_i = int(parse[i].strip('\n').split()[-4])
        idx_j = int(parse[i].strip('\n').split()[-3])
        idx_k = int(parse[i].strip('\n').split()[-2])
        idx_l = int(parse[i].strip('\n').split()[-1])
        
        # get angle energy
        e_torsional = float(parse[i].strip('\n').split()[-5])/4
        
        # add 1/4 of energy if index already exists otherwise initialize key and assign energy
        if idx_i in prop_dict.keys():
            prop_dict[idx_i] += e_torsional
        else:
            prop_dict[idx_i] = e_torsional
            
        if idx_j in prop_dict.keys():
            prop_dict[idx_j] += e_torsional
        else:
            prop_dict[idx_j] = e_torsional
            
        if idx_k in prop_dict.keys():
            prop_dict[idx_k] += e_torsional
        else:
            prop_dict[idx_k] = e_torsional
            
        if idx_l in prop_dict.keys():
            prop_dict[idx_l] += e_torsional
        else:
            prop_dict[idx_l] = e_torsional

        i += 1
    return(sort_dict(prop_dict))

def parse_e_vdw(file):
    """
    get bond energies from output file
    """
    # find section for bond stretching because here I also write positions
    linenumber = 'Nan'
    for ln, line in enumerate(file):
        if line == 'V A N   D E R   W A A L S\n':
            linenumber = ln
            break

    # parse section for positions
    parse = file[linenumber+5:]
    i = 0

    prop_dict = dict()
    while 'TOTAL VAN DER WAALS ENERGY' not in parse[i]:
        # get index of atom i
        idx_i = int(parse[i].strip('\n').split()[-2])
        # get index of atom j
        idx_j = int(parse[i].strip('\n').split()[-1])
        # get vdw energy
        e_vdw = float(parse[i].strip('\n').split()[-3])/2
        
        # add half energy if index already exists otherwise initialize key and assign energy
        if idx_i in prop_dict.keys():
            prop_dict[idx_i] += e_vdw
        else:
            prop_dict[idx_i] = e_vdw
            
        if idx_j in prop_dict.keys():
            prop_dict[idx_j] += e_vdw
        else:
            prop_dict[idx_j] = e_vdw

        i += 1
    return(sort_dict(prop_dict))

In [11]:
with open('/home/sahre/energy_ob', 'r') as f:
    file = f.readlines()

In [25]:
parse_positions(file)

{1: [0.445, -1.095, -0.367],
 2: [0.758, 0.407, -0.289],
 3: [-0.474, 1.307, -0.289],
 4: [-0.282, -1.661, 0.859],
 5: [-0.021, 2.657, -0.289],
 6: [1.39, -1.636, -0.501],
 7: [-0.148, -1.297, -1.271],
 8: [1.345, 0.624, 0.613],
 9: [1.381, 0.699, -1.144],
 10: [-1.101, 1.104, 0.594],
 11: [-1.088, 1.094, -1.181],
 12: [0.288, -1.473, 1.777],
 13: [-0.418, -2.745, 0.768],
 14: [-1.276, -1.217, 0.99],
 15: [-0.8, 3.232, -0.269]}

In [26]:
parse_e_bond(file)

{1: 0.8180000000000001,
 2: 0.6415,
 3: 1.1945000000000001,
 4: 0.6455000000000001,
 5: 1.5805,
 6: 0.0945,
 7: 0.0645,
 8: 0.0845,
 9: 0.1025,
 10: 0.0395,
 11: 0.02,
 12: 0.1035,
 13: 0.125,
 14: 0.124,
 15: 0.534}

In [17]:
parse_e_angle(file)

{1: 2.7766666666666664,
 2: 2.7816666666666663,
 3: 2.066,
 4: 1.662,
 5: 0.7883333333333333,
 6: 0.26033333333333336,
 7: 0.21466666666666667,
 8: 0.17733333333333332,
 9: 0.23166666666666666,
 10: 0.17666666666666667,
 11: 0.17933333333333334,
 12: 0.17666666666666667,
 13: 0.18133333333333332,
 14: 0.29933333333333334,
 15: 0.41300000000000003}

In [24]:
parse_e_torsional(file)

{1: 0.63725,
 2: 2.8035,
 3: 3.592,
 4: 0.31425,
 5: 3.3207500000000003,
 6: 0.357,
 7: 0.03,
 8: 0.06975,
 9: 0.32025000000000003,
 10: 0.6465000000000001,
 11: 0.5810000000000001,
 12: 0.051750000000000004,
 13: 0.05875,
 14: 0.058249999999999996,
 15: 3.265}

In [31]:
parse_e_vdw(file)

{1: 0.3555,
 2: 0.08249999999999999,
 3: 0.804,
 4: 0.786,
 5: 0.7405000000000002,
 6: 0.1285,
 7: 0.20649999999999996,
 8: 0.5255000000000001,
 9: 0.6965,
 10: 1.113,
 11: 0.5675,
 12: -0.11800000000000001,
 13: -0.10900000000000001,
 14: 0.291,
 15: 0.749}

In [32]:
e_bond = parse_e_bond(file)
e_angle = parse_e_angle(file)
e_torsional = parse_e_torsional(file)
e_vdw = parse_e_vdw(file)
ebt = 0 
eat = 0
ett = 0
evt = 0
for k in e_bond.keys():
    ebt += e_bond[k]
    eat += e_angle[k]
    ett += e_torsional[k]
    evt += e_vdw[k]

In [33]:
print(ebt, eat, ett, evt)

6.172 12.384999999999998 16.105999999999998 6.819000000000001


In [ ]:
butanol_example = ase.io.read('/data/sahre/projects/atomic-energies/data/UFF_energies/butanol_ang_1.42429.xyz')

In [ ]:
np.round(np.triu(butanol_example.get_all_distances()), 3)[0]

In [ ]:
butanol_example.get_atomic_numbers()

In [ ]:
molecule = butanol_example.copy()    

In [ ]:
bond_pairs = {}
for idx1 in range(len(molecule.get_atomic_numbers())):
    for idx2 in range(len(molecule.get_atomic_numbers())):
        if idx1 != idx2:
            an1 = molecule.get_atomic_numbers()[idx1]
            an2 =  molecule.get_atomic_numbers()[idx2]
            bond = molecule.get_distance(idx1, idx2)

In [ ]:
def get_r_BO(rI, rJ, n, lambda_=0.1332):
    return(lambda_*(rI+rJ)*np.log(n))
def get_r_EN(rI, rJ, chiI, chiJ):
    return(rI*rJ*(np.sqrt(chiI)-np.sqrt(chiJ))**2/(chiI*rI+chiJ*rJ))

def get_r_IJ(rI, rJ, n, chiI, chiJ, lambda_=0.1332):
    return(rI + rJ + get_r_BO(rI, rJ, n, lambda_)+get_r_EN(rI, rJ, chiI, chiJ))

def get_k_IJ(r_IJ, ZI, ZJ):
    return(664.12*ZI*ZJ/r_IJ**3)

def get_harmonic_bond(r, r_IJ, k_IJ):
    return(0.5*k_IJ*(r-r_IJ)**2)

In [ ]:
rC_3 = 0.757
rO_3 = 0.658
chi_C_3 = 5.343
chi_O_3 = 8.741
n = 1
r_IJ = get_r_IJ(rC_3, rO_3, n, chi_C_3, chi_O_3)
print(r_IJ)
ZO = 2.3
ZC = 1.912
k_IJ = get_k_IJ(r_IJ, ZC, ZO)
print(k_IJ)

In [ ]:
get_r_EN(1.117, 0.528, 4.168, 8.741)

In [ ]:
r = []
e = []
with open('/data/sahre/projects/atomic-energies/data/UFF_energies/CO_bond', 'r') as f:
    for line in f:
        e.append(float(line.split()[-1]))
        r.append(float(line.split()[3]))
r = np.array(r)

In [ ]:
plt.plot(r,get_harmonic_bond(np.array(r), r_IJ, k_IJ))

plt.plot(r,e, '-o')

In [ ]:
plt.plot(r**2,get_harmonic_bond(np.array(r), r_IJ, k_IJ))

In [ ]:
plt.plot(r,e)

In [ ]:
plt.plot(r**2,e)

In [ ]:
butanol = ase.io.read('/data/sahre/projects/atomic-energies/data/UFF_energies/butanol_ang_0.92429.xyz')

In [ ]:
for i in range(30):
    butanol.set_distance(2, 4, 0.05, fix=0, add = True, indices = [4,14])
    d = np.round(butanol.get_distance(2,4), 2)
    print(d)
    ase.io.write(f'/data/sahre/projects/atomic-energies/data/UFF_energies/butanol_d_{d}.xyz', butanol)

In [ ]:
len(butanol.get_atomic_numbers())

In [ ]:
butanol.get_all_distances()[4]


In [ ]:
butanol.get_all_distances()[4]


In [ ]:
ase.visualize.view(butanol)